In [3]:
from pathlib import Path
from collections import OrderedDict as odict

import torch
from torch.utils.data import DataLoader
from torch import nn, optim
from torchvision import transforms, models

from utils import ProteinDataset

In [4]:
def create_dataloader(root_dir):
    data_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor()
    ])

    dataset = ProteinDataset(root_dir / 'train.csv', root_dir / 'train', colors=['red', 'green', 'blue'],
                             transforms=data_transform)
    
    dataloader = DataLoader(dataset, batch_size=16, num_workers=4, shuffle=True)
    
    return dataloader

def create_model(arch, device_id):
    
    class Flatten(nn.Module):
        "Flatten `x` to a single dimension, often used at the end of a model."
        def __init__(self):
            super().__init__()

        def forward(self, x):
            return x.view((x.size(0), -1)) 
    
    body = nn.Sequential(*list(arch.children())[:-1])
    head = nn.Sequential(Flatten(), nn.Linear(2048, 28))

    model = nn.Sequential(odict([('body', body), ('head', head)]))
    
    model.to(device_id)
    
    return model

def create_optimizer(model, lrs):
    assert len(lrs) == 2
    optimizer = optim.SGD([{'params': model.body.parameters(), 'lr': lrs[0]}, 
                       {'params': model.head.parameters(), 'lr': lrs[1]}],
                       momentum=0.9)
    return optimizer

def create_lr_scheduler(optimizer, num_epochs):
    return optim.lr_scheduler.LambdaLR(optimizer, lambda epoch: (1-epoch/num_epochs))

In [5]:
root_dir = Path('/home/xmiler/projects/human-protein-atlas-image-classification/input/')
device_id = 0
arch = models.resnet50(pretrained=True)

dataloader = create_dataloader(root_dir)
model = create_model(arch, device_id)
criterion = nn.MultiLabelSoftMarginLoss()

In [6]:
num_epochs = 1
lrs = [0., 0.01]

optimizer = create_optimizer(model, lrs)
lr_scheduler = create_lr_scheduler(optimizer, num_epochs)

In [7]:
%%time

i = 0

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    
    lr_scheduler.step()
    
    for inputs, labels in dataloader:
        inputs = inputs.to(device_id)
        labels = labels.float().to(device_id)
        
        optimizer.zero_grad()
        
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
        i += 1
        
        if i == 100:
            break

Epoch 0/0
----------
CPU times: user 9.99 s, sys: 3.77 s, total: 13.8 s
Wall time: 14.1 s


In [6]:
%%time

i = 0

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    
    lr_scheduler.step()
    
    for inputs, labels in dataloader:
        inputs = inputs.to(device_id)
        labels = labels.float().to(device_id)
        
        optimizer.zero_grad()
        
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
        i += 1
        
        if i == 100:
            break

Epoch 0/0
----------
CPU times: user 16.8 s, sys: 1.5 s, total: 18.3 s
Wall time: 20 s


In [8]:
loss

tensor([0.6429, 0.6432, 0.6538, 0.6752, 0.6630, 0.6697, 0.6534, 0.6529, 0.6739,
        0.6812, 0.6675, 0.6877, 0.6351, 0.6502, 0.6799, 0.6549],
       device='cuda:0', grad_fn=<DivBackward0>)

In [21]:
class Learner(object):
    def __init__(self, dataloader, model):
        self._model = model

In [12]:
learner = Learner(model)

learner

In [11]:
%config IPCompleter.greedy=True

In [10]:
inputs.shape

torch.Size([16, 3, 224, 224])

In [13]:
a = model[1]
list(a.named_parameters())

[('1.weight', Parameter containing:
  tensor([[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]], device='cuda:0')),
 ('1.bias', Parameter containing:
  tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0'))]

In [11]:
model(inputs)

tensor([[ 984.3602,  984.3602,  984.3602,  984.3602,  984.3602,  984.3602,
          984.3602,  984.3602,  984.3602,  984.3602,  984.3602,  984.3602,
          984.3602,  984.3602,  984.3602,  984.3602,  984.3602,  984.3602,
          984.3602,  984.3602,  984.3602,  984.3602,  984.3602,  984.3602,
          984.3602,  984.3602,  984.3602,  984.3602],
        [ 789.5058,  789.5058,  789.5058,  789.5058,  789.5058,  789.5058,
          789.5058,  789.5058,  789.5058,  789.5058,  789.5058,  789.5058,
          789.5058,  789.5058,  789.5058,  789.5058,  789.5058,  789.5058,
          789.5058,  789.5058,  789.5058,  789.5058,  789.5058,  789.5058,
          789.5058,  789.5058,  789.5058,  789.5058],
        [1000.3049, 1000.3049, 1000.3049, 1000.3049, 1000.3049, 1000.3049,
         1000.3049, 1000.3049, 1000.3049, 1000.3049, 1000.3049, 1000.3049,
         1000.3049, 1000.3049, 1000.3049, 1000.3049, 1000.3049, 1000.3049,
         1000.3049, 1000.3049, 1000.3049, 1000.3049, 1000.3049, 100

In [10]:
for n,p in model[1].named_parameters():    
    p.requires_grad=False
    p.zero_()
    p.add_(1.)

In [48]:
list(model[1].parameters())

[Parameter containing:
 tensor([[ 0.0065,  0.0162,  0.0008,  ..., -0.0007,  0.0088,  0.0199],
         [ 0.0014,  0.0108,  0.0013,  ...,  0.0128,  0.0136, -0.0143],
         [ 0.0111, -0.0135, -0.0187,  ...,  0.0101,  0.0097, -0.0176],
         ...,
         [ 0.0117,  0.0012, -0.0014,  ...,  0.0136,  0.0100, -0.0203],
         [-0.0044,  0.0208,  0.0154,  ...,  0.0043, -0.0022, -0.0016],
         [ 0.0189,  0.0063,  0.0081,  ...,  0.0046,  0.0032, -0.0178]],
        device='cuda:0', requires_grad=True), Parameter containing:
 tensor([-0.0074,  0.0008, -0.0100,  0.0185, -0.0091,  0.0027, -0.0005, -0.0007,
          0.0039, -0.0182,  0.0052,  0.0001,  0.0020,  0.0053,  0.0140, -0.0090,
         -0.0050, -0.0128,  0.0148, -0.0179,  0.0125,  0.0214,  0.0207,  0.0007,
         -0.0009,  0.0099, -0.0030, -0.0009],
        device='cuda:0', requires_grad=True)]

In [15]:
c1(d)

RuntimeError: size mismatch, m1: [32768 x 1], m2: [2048 x 28] at /pytorch/aten/src/THC/generic/THCTensorMathBlas.cu:249

In [7]:
model.fc = nn.Linear(model.fc.in_features, 28)

In [16]:
model.fc.in_features

2048

In [8]:
model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [12]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [9]:
for n,p in model.named_parameters():
    if n.startswith('fc'):
        p.requires_grad=False
        p.zero_()
        p.add_(1.)

In [10]:
p

Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0')

In [11]:
model(inputs)

tensor([[ 984.3602,  984.3602,  984.3602,  984.3602,  984.3602,  984.3602,
          984.3602,  984.3602,  984.3602,  984.3602,  984.3602,  984.3602,
          984.3602,  984.3602,  984.3602,  984.3602,  984.3602,  984.3602,
          984.3602,  984.3602,  984.3602,  984.3602,  984.3602,  984.3602,
          984.3602,  984.3602,  984.3602,  984.3602],
        [ 789.5058,  789.5058,  789.5058,  789.5058,  789.5058,  789.5058,
          789.5058,  789.5058,  789.5058,  789.5058,  789.5058,  789.5058,
          789.5058,  789.5058,  789.5058,  789.5058,  789.5058,  789.5058,
          789.5058,  789.5058,  789.5058,  789.5058,  789.5058,  789.5058,
          789.5058,  789.5058,  789.5058,  789.5058],
        [1000.3049, 1000.3049, 1000.3049, 1000.3049, 1000.3049, 1000.3049,
         1000.3049, 1000.3049, 1000.3049, 1000.3049, 1000.3049, 1000.3049,
         1000.3049, 1000.3049, 1000.3049, 1000.3049, 1000.3049, 1000.3049,
         1000.3049, 1000.3049, 1000.3049, 1000.3049, 1000.3049, 100

In [69]:
for p, _ in model.named_parameters():
    print(p)

conv1.weight
bn1.weight
bn1.bias
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.conv3.weight
layer1.0.bn3.weight
layer1.0.bn3.bias
layer1.0.downsample.0.weight
layer1.0.downsample.1.weight
layer1.0.downsample.1.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.1.conv3.weight
layer1.1.bn3.weight
layer1.1.bn3.bias
layer1.2.conv1.weight
layer1.2.bn1.weight
layer1.2.bn1.bias
layer1.2.conv2.weight
layer1.2.bn2.weight
layer1.2.bn2.bias
layer1.2.conv3.weight
layer1.2.bn3.weight
layer1.2.bn3.bias
layer2.0.conv1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.conv2.weight
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.0.conv3.weight
layer2.0.bn3.weight
layer2.0.bn3.bias
layer2.0.downsample.0.weight
layer2.0.downsample.1.weight
layer2.0.downsample.1.bias
layer2.1.conv1.weight
layer2.1.bn1.weight
layer2.1.bn1.bias
layer2.1.conv2.we

In [65]:
for i, (a,b) in enumerate(model.named_children()):
    print(a)
    if i == 4:
        break

conv1
bn1
relu
maxpool
layer1


In [68]:
for a in b.named_parameters():
    print (a)

('0.conv1.weight', Parameter containing:
tensor([[[[ 0.0035]],

         [[ 0.0399]],

         [[-0.0248]],

         ...,

         [[ 0.1028]],

         [[-0.2893]],

         [[-0.1518]]],


        [[[ 0.1149]],

         [[ 0.0057]],

         [[ 0.0284]],

         ...,

         [[ 0.0807]],

         [[-0.0848]],

         [[-0.1358]]],


        [[[-0.0190]],

         [[ 0.0112]],

         [[-0.0160]],

         ...,

         [[ 0.0715]],

         [[ 0.0767]],

         [[-0.0460]]],


        ...,


        [[[-0.0020]],

         [[-0.0019]],

         [[ 0.0237]],

         ...,

         [[ 0.0161]],

         [[-0.0623]],

         [[ 0.0094]]],


        [[[-0.0528]],

         [[ 0.0107]],

         [[ 0.0320]],

         ...,

         [[-0.0219]],

         [[-0.0061]],

         [[ 0.0126]]],


        [[[-0.0565]],

         [[-0.1190]],

         [[ 0.0471]],

         ...,

         [[ 0.0410]],

         [[ 0.2449]],

         [[-0.0079]]]], device='cuda:0'

In [55]:
for param in model.named_parameters():
    break
    
param

('conv1.weight', Parameter containing:
 tensor([[[[ 0.0133,  0.0147, -0.0154,  ..., -0.0409, -0.0430, -0.0708],
           [ 0.0041,  0.0058,  0.0149,  ...,  0.0022, -0.0209, -0.0385],
           [ 0.0223,  0.0236,  0.0161,  ...,  0.1028,  0.0626,  0.0520],
           ...,
           [-0.0009,  0.0278, -0.0101,  ..., -0.1272, -0.0766,  0.0078],
           [ 0.0036,  0.0480,  0.0621,  ...,  0.0243, -0.0337, -0.0157],
           [-0.0800, -0.0322, -0.0178,  ...,  0.0354,  0.0224,  0.0017]],
 
          [[-0.0185,  0.0114,  0.0239,  ...,  0.0537,  0.0440, -0.0095],
           [-0.0077,  0.0189,  0.0680,  ...,  0.1596,  0.1461,  0.1200],
           [-0.0460, -0.0761, -0.0896,  ...,  0.1211,  0.1670,  0.1762],
           ...,
           [ 0.0288,  0.0137, -0.0838,  ..., -0.3808, -0.3041, -0.1397],
           [ 0.0829,  0.1386,  0.1524,  ..., -0.0051, -0.1244, -0.1297],
           [-0.0073,  0.0770,  0.1400,  ...,  0.1843,  0.1114,  0.0234]],
 
          [[-0.0183, -0.0056,  0.0087,  ...,  0

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from skimage import io, transform
from sklearn.preprocessing import MultiLabelBinarizer
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [2]:
IMG_STATS = ([0.08069, 0.05258, 0.05487, 0.08282], [0.13704, 0.10145, 0.15313, 0.13814])

def open_image(image_name, colors, size):
    image_name = str(image_name)
    img = [io.imread(image_name+'_'+color+'.png').astype(np.float32)/255 for color in colors]    
    if size is not None:
        img = [transform.resize(c, (size, size), mode='reflect', anti_aliasing=False) for c in img]
    img = np.stack(img, axis=-1)
    return img

In [3]:
root_dir = Path('/home/xmiler/projects/human-protein-atlas-image-classification/input/')

class ProteinDataset(Dataset):
    def __init__(self, csv_file, images_dir,
                 colors=['red', 'green', 'blue'], size=None, transforms=None):
        csv_content = pd.read_csv(csv_file)
        self._filenames = csv_content['Id'].tolist()        
        self._labels = MultiLabelBinarizer().fit_transform([tuple(int(i) for i in item.split(' ')) for item in  csv_content['Target'].tolist()])
        assert len(self._filenames) == len(self._labels)
        self._images_dir = images_dir
        self._colors = colors
        self._size = size
        self._transforms = transforms
        
    def __len__(self):
        return len(self._filenames)
    
    def __getitem__(self, idx):
        image = open_image(self._images_dir / self._filenames[idx], 
                           self._colors, self._size)
        if self._transforms is not None:
            image = self._transforms(image)
        
        labels = self._labels[idx]
        
        sample = {'image': image, 'labels': labels}        
        
        return sample

In [4]:
data_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize(*IMG_STATS)])

In [5]:
dataset = ProteinDataset(root_dir / 'train.csv', root_dir / 'train', size=224, transforms=data_transform)

In [8]:
i = 103
print(dataset[i]['image'][0].mean())
print(dataset[i]['image'][0].std())

tensor(0.0396, dtype=torch.float64)
tensor(0.8022, dtype=torch.float64)


In [10]:
dataloader = DataLoader(dataset, batch_size=16, num_workers=2)

In [11]:
for batch_i, batch in enumerate(dataloader):
    break

In [16]:
batch['image'].size()

torch.Size([16, 3, 224, 224])